# Text Analysis Assignment

## Assignment Details

The text below is a summary from [this document](https://docs.google.com/document/d/1FNRmAS_vc-2eQESHH6uZMcuDVr720rsx-aDWGH3oz_Y/edit?usp=sharing).

The main goal of the assignment is to have you practice the tools we have been using in class.

The requirements are:

- Choose one or more texts to work with.
- Either save the text files in your working directory, or have python get them from a web address.
- If needed: convert your text from bytes to a string
- Tokenize your text
- Make it an NLTK text object so you can use nltk tools on it
- Clean the text in a way that is appropriate for the kind of analysis you want to do.
- Run some analysis
- Report findings

Important notes:

- If you plan to use functions have your functions as a separate python file, and import it in your main file.

- Please, over-comment your script. Make sure to comment every step of the way. Make sure to not only explain what you are doing in terms of code, but also your analytical goal too. For instance, both "I am running a for loop to remove the common words" and "I am trying to see how these two authors compare in terms of the ratio of unusual words to total words" are kind of comments I want to see.


## Assignment Submission

### Itinerary

I want to compare usage of modal verbs in British men and women authors in the 18th and 19th century. To do my proposed analysis, I will

- Download a sample of raw texts from [Project Gutenberg](https://www.gutenberg.org/) using `urllib`
- Convert the texts from bytes to a string
- Tokenize the texts
- Make it an NLTK text object
- Clean the texts, including removing front matter and other empherma from Project Gutenberg texts
- Create a list of modal verbs, filter for these works in each text
- Perhaps create a conditional frequency distribution on all texts (?) to see if we can establish a pattern

### Imports

In the cell below, I'm importing the libraries/modules:

In [3]:
from urllib.request import urlopen
from nltk.probability import FreqDist
from nltk.probability import ConditionalFreqDist